# Rule Splitting & ElasticSearch Uploads
## LMI Capstone Team
## Summer Chambers | Steve Morris | Kaleb Shikur

### Setup

In [131]:
import pandas as pd
import elasticsearch
from elasticsearch import Elasticsearch, RequestsHttpConnection, ElasticsearchException
import regex as re
from bs4 import BeautifulSoup
import numpy as np
import requests #gets urls
import time
import json
import csv
import os
import sys

In [6]:
# 173.79.72.92, port 9200
# host = '173.79.72.92'
endpoint = 'https://search-lmi-capstone-2-525zkk33t4z5iy6ozqd63ctgmq.us-east-1.es.amazonaws.com/'

In [55]:
es = Elasticsearch(endpoint, timeout = 45)

print(es.info())

# es = Elasticsearch(
#     hosts=[{'host': host, 'port': 443}],
#     http_auth=auth,
#     use_ssl=True,
#     verify_certs=True,
#     connection_class=RequestsHttpConnection
# )
# print(es.info())

{'name': '615fc8a41521934b0c7f55cbeafff841', 'cluster_name': '846033058400:lmi-capstone-2', 'cluster_uuid': 'QzTffcNgStmet053IRSP2w', 'version': {'number': '7.9.1', 'build_flavor': 'oss', 'build_type': 'tar', 'build_hash': 'unknown', 'build_date': '2020-11-03T09:54:32.349659Z', 'build_snapshot': False, 'lucene_version': '8.6.2', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}


### Define Rule Splits

In [9]:
rule_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Rule/CMS-2018-0101-0001.txt"

In [10]:
def splitRule_headers(rule_url):

    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    rulechunks = {}
    
    startdict = {'a2':['2. proposals for modified participation options under 5-year agreement periods', '3. creating a basic track with glide path to performance-based risk'], \
     'a3':['3. creating a basic track with glide path to performance-based risk', '4. permitting annual participation elections'], \
    'a4b':['b. proposals for permitting election of differing levels of risk within the basic track\'s glide path', 'c. proposals for permitting annual election of beneficiary assignment methodology'], \
    'a4c':['c. proposals for permitting annual election of beneficiary assignment methodology', '5. determining participation options based on medicare ffs revenue and prior participation '], \
    'a5b':['b. differentiating between low revenue acos and high revenue acos', 'c. determining participation options based on prior participation of aco legal entity and aco participants'], \
    'a5c':['c. determining participation options based on prior participation of aco legal entity and aco participants', 'd. monitoring for financial performance'], \
    'a5d': ['d. monitoring for financial performance', '6. requirements for aco participation in two-sided models'], \
    'a6b': ['b. election of msr/mlr by acos', 'c. aco repayment mechanisms'], \
    'a6c': ['c. aco repayment mechanisms', 'd. advance notice for and payment consequences of termination '], \
    'a6d2': ['(2) proposals for advance notice of voluntary termination', '(3) proposals for payment consequences of termination'], \
    'a6d3': ['(3) proposals for payment consequences of termination', '7. participation options for agreement periods beginning in 2019'], \
    'a7b': ['b. methodology for determining financial and quality performance for the 6-month performance years during 2019', 'c. applicability of program policies to acos participating in a 6-month performance year'], \
    'a7c': ['c. applicability of program policies to acos participating in a 6-month performance year', 'b. fee-for-service benefit enhancements'], \
    'b2a': ['a. shared savings program snf 3-day rule waiver', 'b. billing and payment for telehealth services'], \
    'b2b': ['b. billing and payment for telehealth services', 'c. providing tools to strengthen beneficiary engagement'], \
    'c2': ['2. beneficiary incentives', '3. empowering beneficiary choice'], \
    'c3a': ['3. empowering beneficiary choice', 'b. beneficiary opt-in based assignment methodology'], \
    'c3b': ['b. beneficiary opt-in based assignment methodology', 'd. benchmarking methodology refinements'],  \
    'd2': ['2. risk adjustment methodology for adjusting historical benchmark each performance year', '3. use of regional factors when establishing and resetting acos\' benchmarks'], \
    'd3b': ['b. proposals to apply regional expenditures in determining the benchmark for an aco\'s first agreement period', 'c. proposals for modifying the regional adjustment'], \
    'd3c': ['c. proposals for modifying the regional adjustment', 'd. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark'], \
    'd3d': ['d. proposals for modifying the methodology for calculating growth rates used in establishing, resetting, and updating the benchmark', '4. technical changes to incorporate references to benchmark rebasing policies'], \
    'd4': ['4. technical changes to incorporate references to benchmark rebasing policies', 'e. updating program policies'], \
    'e2': ['2. revisions to policies on voluntary alignment', '3. revisions to the definition of primary care services used in beneficiary assignment'], \
    'e3': ['3. revisions to the definition of primary care services used in beneficiary assignment', '4. extreme and uncontrollable circumstances policies for the shared savings program'], \
    'e4': ['4. extreme and uncontrollable circumstances policies for the shared savings program', '5. program data and quality measures'], \
    'e5':['5. program data and quality measures', '6. promoting interoperability'], \
    'e6':['6. promoting interoperability', '7. coordination of pharmacy care for aco beneficiaries'], \
    'e7':['7. coordination of pharmacy care for aco beneficiaries', 'f. applicability of proposed policies to track 1+ model acos'], \
    'f2':['2. unavailability of application cycles for entry into the track 1+ model in 2019 and 2020', '3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements'], \
    'f3':['3. applicability of proposed policies to track 1+ model acos through revised program regulations or revisions to track 1+ model participation agreements', 'g. summary of proposed timing of applicability']}
    
    for key, value in startdict.items():    
       splitlist = splitlist.split(value[0]) #split on start of desired section
       split_further = splitlist[1].split(value[1]) #split again on start of undesired section
       rulechunks[key] = {"text": (value[0]+split_further[0])} #choose only first half to upload to dict
       splitlist = splitlist[1] #choose second half to prepare for next split
        
    #lengths = [len(chunk) for chunk in rulechunks.values()]

    return rulechunks

#Updated Character-Count Split

def splitRule_line_chars(rule_url):
    #get rule
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings

    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #Split on new paragraphs
    paragraphs = splitlist.split('\r\n')
    
    #add new lines while under 2000 characters
    for i in range(len(paragraphs) - 1):
        while i < (len(paragraphs)-1) and len(paragraphs[i]) < 2000:
            paragraphs[i] += paragraphs[i+1]
            del(paragraphs[i+1])

    #add to dictionary
    rulechunks = {}
    keys = range(len(paragraphs))
    for i in keys:
        rulechunks[i] = {"text": paragraphs[i]}

    return rulechunks

splitchars = splitRule_line_chars(rule_url)
lengths = {key: len(value2) for key, value in splitchars.items() for key2, value2 in value.items()}

'''
def splitRule_chars(rule_url):
    alltxt = requests.get(rule_url).text.lower()#.encode('unicode_escape').decode() #encodes like raw strings
    
    #Isolate Section 2
    initialsplit = alltxt.split("ii. provisions of the proposed regulations") #split before section 2
    sec2andon = initialsplit[1] #choose latter half
    sec2list = sec2andon.split("iii. collection of information requirements") #split before section 3
    splitlist = sec2list[0] #choose first half
    
    #chunk by num chars
    #chunklist = []
    
    chunklist = [(splitlist[i:i+8000]) for i in range(0, len(splitlist), 8000)]
    
    #make dict
    rulechunks = {}
    keys = range(len(chunklist))
    for i in keys:
        rulechunks[i] = {"text": chunklist[i]}

    return rulechunks
'''

'''
def splitRule_headchars(rule_url):
    chunks = splitRule_headers(rule_url) #call first method
    for key, value in chunks.items():
        if len(value["text"]) > 50000: #check to see if chunk is really long
            smalldict = {}
            chunklist = [value["text"][i:i+25000] for i in range(0, len(value["text"]), 25000)] #split chunk
            keys = range(len(chunklist))
            for i in keys:
                smalldict[key+str(i)] = {"text": chunklist[i]} #add to sub-dictionary
            chunks[key] = smalldict #add to big dictionary
        else:
            chunks[key] = {key+str(0): value} #add to big dictionary
    return chunks
'''

In [12]:
def splitRule_line_hybrid(rule_url):
    new_rule_chunks = {}
    chunks = splitRule_headers(rule_url)
    for key, value in chunks.items():
        for text, real_value in value.items():
            paragraphs = real_value.split('\r\n')
            #add new lines while under 2000 characters
            for i in range(len(paragraphs) - 1):
                while i < (len(paragraphs)-1) and len(paragraphs[i]) < 6000:
                    paragraphs[i] += paragraphs[i+1]
                    del(paragraphs[i+1])
            for i in range(len(paragraphs)):
                new_rule_chunks[key+str(i)] = {text: paragraphs[i]}
    return new_rule_chunks

In [13]:
result = splitRule_line_hybrid(rule_url)
lengths = {key: len(small_value) for key, value in result.items() for small_key, small_value in value.items()}

### Uploading Rule Splits to ES

In [14]:
##es.indices.refresh(index="hybrid_4000")

In [265]:
def rulesplit_toES(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
            res = es.index(index=es_index, id=idnum, body={key:value}, doc_type='_doc')
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")

'''
def ruletoES_hybrid(rulechunks, es_index, idnum):
    for key, value in rulechunks.items():
        for small_key, small_value in value.items():
            res = es.index(index=es_index, id=idnum, body={small_key:small_value})
            idnum += 1
            es.indices.refresh(index=es_index)
    print(f"Last id uploaded: {idnum-1}")
'''

In [ ]:
rulesplit_test1 = splitRule_headers(rule_url)
test_upload1 = rulesplit_toES(rulesplit_test1, "headers", 1)

In [ ]:
rulesplit_test2 = splitRule_line_hybrid(rule_url)
test_upload2 = rulesplit_toES(rulesplit_test2, "hybrid_6000", 1)

### More Utilities:

Delete an index:
es.indices.delete(index='index_name', ignore=[400, 404])

## Retrieve Comment Documents

### Scrape comments

In [122]:
comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"

In [123]:
def retrieve_comments(comment_url):
    comment_url = "https://mikeanders.org/data/CMS/CMS-2018-0101-0001/Comments/"
    response = requests.get(comment_url)
    soup = BeautifulSoup(response.text, 'html.parser')
    a_tags = soup.findAll("a")
    links = [tag["href"] for tag in a_tags]
    txt_links = [link for link in links if '.txt' in link]
    comments = {}
    for suffix in txt_links:
        comments[suffix] = requests.get(comment_url+suffix).text.lower()
        #print(f"scraping comment {suffix}")
    print(f"scraped {len(txt_links)} comments")
    return comments

In [124]:
comments2018 = retrieve_comments(comment_url)

scraped 674 comments


In [125]:
comment_lengths = {key: len(value) for key, value in comments2018.items()}
sorted_comment_lengths = dict(sorted(comment_lengths.items(), key=lambda x: x[1]))

In [126]:
comments2018 = {(key[14:20]): value for key, value in comments2018.items()}
sorted_keys = sorted(list(comments2018.keys()))

# now for each key in the list
for i in range(len(comments2018)-1):
    # get key at index i and key at index i+1 and compare them
    if sorted_keys[i+1][0:4] == sorted_keys[i][0:4]:
        comments2018[sorted_keys[i+1]] = comments2018[sorted_keys[i]] + comments2018[sorted_keys[i+1]]
        del(comments2018[sorted_keys[i]])

In [127]:
comments2018 = {key[0:4]:value for key, value in comments2018.items()}

In [128]:
len(comments2018)

468

### Write to .json

In [132]:
comment_json = json.dumps(comments2018)
f = open('comments2018.json','w')
f.write(comment_json)
f.close()

## Read .json (avoid scraping comments)

In [133]:
with open('comments2018.json') as f:
    comments2018 = json.load(f)

### Remove duplicate comments, comments less than 30 characters long

In [134]:
reduced_comments2018 = {key: comment for key, comment in comments2018.items() if len(comment) >= 30}
short_ones = {key: comment for key, comment in comments2018.items() if len(comment) < 30}
list_reduced_comments2018 = list(reduced_comments2018.values())
len(reduced_comments2018)
short_ones #these seem okay to remove

{'0083': 'good luck everyone!',
 '0142': 'see attached file(s)',
 '0237': 'see attached file(s)\n',
 '0270': 'see attached file(s)\n',
 '0293': 'see attached file(s)',
 '0321': 'see attached file(s)\n\n',
 '0324': 'see attachement\n',
 '0329': 'see attached file(s)\n',
 '0342': 'see attached file(s)\n',
 '0447': 'see attached file(s)'}

In [24]:
#from DeDuper import *

In [138]:
from Levenshtein import distance
def getDupes(texts):
    toProcess = [(i, t) for i, t in enumerate(texts)]
    groups = []
    curIdx, curText = toProcess.pop()
    while toProcess:
        toComb = []
        for i, (compIdx, compText) in enumerate([tup for tup in toProcess]):
            targetLen = max(len(curText), len(compText)) if not len(curText) < 60 and len(compText) < 60 else min(len(curText), len(compText))
            textDist = distance(curText, compText)
            if textDist < (targetLen * 0.30) + max(0, (targetLen/2) - 30):
                toComb.append((i, compIdx, textDist))
        if toComb:
            newGroup = [curIdx]
            for idx, _, _ in toComb:
                try:
                    newGroup.append(toProcess.pop(idx)[0])
                except Exception as e:
                    print(e, "at", idx)
                    continue
            groups.append(newGroup)
        if toProcess:
            curIdx, curText = toProcess.pop()
    return groups

In [139]:
duplicates = getDupes(list_reduced_comments2018)

pop index out of range at 450
pop index out of range at 231
pop index out of range at 232
pop index out of range at 234
pop index out of range at 235
pop index out of range at 236
pop index out of range at 237
pop index out of range at 239
pop index out of range at 240
pop index out of range at 248
pop index out of range at 253
pop index out of range at 262
pop index out of range at 263
pop index out of range at 362
pop index out of range at 364
pop index out of range at 365
pop index out of range at 143
pop index out of range at 220
pop index out of range at 222
pop index out of range at 138
pop index out of range at 57
pop index out of range at 17
pop index out of range at 18
pop index out of range at 19
pop index out of range at 12
pop index out of range at 13


In [140]:
len(duplicates)

29

In [141]:
tb_deleted = []
for num, duplist in enumerate(duplicates):
    for idx, comment in enumerate(list_reduced_comments2018):
        if idx in duplist[:-1]:
            tb_deleted.append(comment)

In [142]:
len(tb_deleted)

403

In [143]:
unique_comments2018 = {key:value for key, value in reduced_comments2018.items() if value not in tb_deleted}

In [144]:
len(unique_comments2018)

53

### Testing other basic API Features

Check to see if document_id exists in index

In [146]:
es.exists(index="headers", id=17)

True

View term vectors for a document

In [151]:
es.termvectors(index="headers", id=6)

{'_index': 'headers',
 '_type': '_doc',
 '_id': '6',
 '_version': 1,
 'found': True,
 'took': 1,
 'term_vectors': {}}

See an explanation for a query's score

In [ ]:
es.explain(index="hybrid_6000", id=6, body={"query": test_comment0002_query}) 

### Test-Querying ES

In [18]:
#Single query
def ES_search(es_index, querydict):
    search = es.search(index=es_index, doc_type="_doc", body={"query": querydict})
    test_dict = {}
    if search['hits']['hits'] != []:
        for h in search['hits']['hits']:
            key = list((h['_source'].keys()))
            test_dict[key[0]]=h['_score']
    return test_dict

With "Simple Query String"

In [162]:
test_comment0002_query = {"simple_query_string": {"query": comments2018["0002"]}}

In [168]:
ES_search("hybrid_6000", test_comment0002_query)

{'a20': 84.076,
 'a6c42': 81.94669,
 'a4c72': 81.94669,
 'a6c76': 81.30872,
 'a4c106': 81.30872,
 'a4c35': 80.02778,
 'a5c6': 79.36414,
 'a4c16': 79.36414,
 'a6c23': 78.574585,
 'a4c53': 78.574585}

With "Query String"

In [263]:
query_string_test0002 = {"query_string": {"query": comments2018["0002"], "minimum_should_match": "70%"}}

In [264]:
ES_search("hybrid_6000", query_string_test0002)

{'a20': 82.25747,
 'a6c76': 80.29602,
 'a4c106': 80.29602,
 'a31': 77.03246,
 'a4c30': 76.39613,
 'a4c48': 76.18532,
 'a6c18': 76.18532,
 'a4c102': 75.71008,
 'a6c72': 75.71008,
 'e60': 75.009}

With "More Like This"

In [218]:
mlt_test0002 = {"more_like_this": {"fields": ["text"], "like": comments2018["0002"], "min_term_freq": 1, "max_query_terms": 20}}

In [219]:
ES_search("hybrid_6000", mlt_test0002)

{}

With "Match" or "Match_Phrase"

In [230]:
match_test0002 = {"match_phrase" : {"text" : comments2018["0002"]}}

In [231]:
ES_search("hybrid_6000", match_test0002)

{}

### Querying Many Comments

In [191]:
es.indices.refresh(index='hybrid_6000')

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}

In [159]:
def run_large_query(comment_dict, es_index):
    results = {}
    for key, value in comment_dict.items():
        query = {"simple_query_string": {"query": value}}
        try:
            search1 = ES_search(es_index, query)
        except ElasticsearchException as es1:
            print(f'max clause error at {key}', end=' ')
            #print(es1)
        results[key] = list(search1.keys())
        time.sleep(1)
        es.indices.refresh(index=es_index)
        print(key, end=' ')
    return results

In [160]:
# dict_first3 = {key:value for key,value in comments2018}
dict_first3 = {}
for key in list(comments2018.keys())[0:3]:
    dict_first3[key] = comments2018[key]

In [ ]:
# Test CHARACTERS on dictionary of 3 key/value
run_large_query(dict_first3, "char_25000")

In [ ]:
# Test HEADERS on dictionary of 3 key/value
run_large_query(dict_first3, "headers_new")

In [161]:
# Test HYBRID on dictionary of 3 key/value
run_large_query(dict_first3, "hybrid_6000")

0002 max clause error at 0003 0003 0004 

{'0002': ['a20',
  'a6c42',
  'a4c72',
  'a6c76',
  'a4c106',
  'a4c35',
  'a5c6',
  'a4c16',
  'a6c23',
  'a4c53'],
 '0003': ['a20',
  'a6c42',
  'a4c72',
  'a6c76',
  'a4c106',
  'a4c35',
  'a5c6',
  'a4c16',
  'a6c23',
  'a4c53'],
 '0004': ['a4c102',
  'a6c72',
  'a4c26',
  'a6c17',
  'a4c47',
  'a32',
  'a4c48',
  'a6c18',
  'a7c0',
  'a6c42']}

#### CHARCTER Full dictionary Query

In [ ]:
# CHARACTER Full dictionary Query
char_results = run_large_query(comments2018, "char_25000")

In [ ]:
# Capture CHARACTER results in CSV
w = csv.writer(open('char_results.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
# Capture CHARACTER results in JSON
json = json.dumps(results)
f = open('char_results.json','w')
f.write(json)
f.close()

#### HEADERS Full dictionary Query

In [ ]:
header_results = run_large_query(comments2018, "headers_new")

In [ ]:
w = csv.writer(open('header_results2.csv', 'w'))
for key, val in results.items():
    w.writerow([key,val])

In [ ]:
json = json.dumps(results)
f = open('header_results2.json','w')
f.write(json)
f.close()

#### HYBRID Full dictionary Query

In [ ]:
hybrid_results = run_large_query(comments2018, "hybrid_6000")

In [ ]:
# Capture HYBRID results in CSV
w = csv.writer(open('hybrid_results.csv', 'w'))
for key, val in hybrid_results.items():
    w.writerow([key,val])

In [ ]:
# Capture HYBRID results in JSON
json = json.dumps(hybrid_results)
f = open('hybrid_results.json','w')
f.write(json)
f.close()